In [7]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin_min

# If Using Google Colab, enable these extensions
# from google.colab import output
# output.enable_custom_widget_manager()

class InteractiveKMeans:
    def __init__(self):
        self.fig = Figure(figsize=(12, 8))
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasAgg(self.fig)
        self.scatter = None
        self.centroid_scatter = None
        self.create_synthetic_data()
        self.create_widgets()
        self.reset()

    def create_synthetic_data(self):
        np.random.seed(42)
        n_samples = 1000
        n_features = 2
        n_clusters = 5

        # Create cluster centers
        centers = np.array([
            [-5, -5],
            [0, 5],
            [5, -3],
            [-3, 3],
            [5, 5]
        ])

        # Generate data points around each center
        X = np.vstack([
            np.random.randn(n_samples // n_clusters, n_features) + center
            for center in centers
        ])

        # Shuffle the data
        np.random.shuffle(X)

        self.data = X
        self.true_labels = np.repeat(range(n_clusters), n_samples // n_clusters)

    def create_widgets(self):
        self.play_button = widgets.Play(value=0, min=0, max=100, step=1, interval=1000, description="Play")
        self.slider = widgets.IntSlider(min=0, max=100, step=1, description='Iteration:')
        widgets.jslink((self.play_button, 'value'), (self.slider, 'value'))

        self.n_clusters = widgets.IntSlider(value=5, min=2, max=7, step=1, description='Clusters:')

        self.reset_button = widgets.Button(description="Reset")
        self.reset_button.on_click(self.reset)
        self.play_button.observe(self.step, 'value')
        
        self.output = widgets.Output()
        self.info_output = widgets.Output()

    def reset(self, _=None):
        self.centroids = self.data[np.random.choice(len(self.data), self.n_clusters.value, replace=False)]
        self.labels = np.zeros(len(self.data), dtype=int)
        self.iteration = 0
        self.update_plot()
        self.play_button.value = 0

    def assign_labels(self):
        distances = np.sqrt(((self.data - self.centroids[:, np.newaxis])**2).sum(axis=2))
        self.labels = np.argmin(distances, axis=0)

    def update_centroids(self):
        for i in range(self.n_clusters.value):
            if np.sum(self.labels == i) > 0:
                self.centroids[i] = np.mean(self.data[self.labels == i], axis=0)

    def step(self, _):
        self.assign_labels()
        self.update_centroids()
        self.iteration += 1
        self.update_plot()

    def calculate_centroid_stats(self):
        distances, _ = pairwise_distances_argmin_min(self.data, self.centroids)
        stats = []
        for i in range(self.n_clusters.value):
            cluster_points = self.data[self.labels == i]
            cluster_distances = distances[self.labels == i]
            if len(cluster_points) > 0:
                stats.append({
                    'centroid': self.centroids[i],
                    'size': len(cluster_points),
                    'avg_distance': np.mean(cluster_distances),
                    'max_distance': np.max(cluster_distances)
                })
            else:
                stats.append({
                    'centroid': self.centroids[i],
                    'size': 0,
                    'avg_distance': np.nan,
                    'max_distance': np.nan
                })
        return stats

    def update_plot(self):
        self.ax.clear()
        self.scatter = self.ax.scatter(self.data[:, 0], self.data[:, 1], c=self.labels, cmap='viridis', alpha=0.7)
        self.centroid_scatter = self.ax.scatter(self.centroids[:, 0], self.centroids[:, 1], c='red', marker='x', s=200, linewidths=3)

        self.ax.set_xlabel('Feature 1')
        self.ax.set_ylabel('Feature 2')
        self.ax.set_title(f'K-means Clustering on Synthetic Dataset - Iteration {self.iteration}')

        self.canvas.draw()
        self.output.clear_output(wait=True)
        with self.output:
            display(self.fig)

        stats = self.calculate_centroid_stats()
        self.info_output.clear_output(wait=True)
        with self.info_output:
            print(f"Iteration: {self.iteration}")
            for i, stat in enumerate(stats):
                print(f"\nCluster {i + 1}:")
                print(f"  Centroid: ({stat['centroid'][0]:.2f}, {stat['centroid'][1]:.2f})")
                print(f"  Size: {stat['size']}")

    def run(self):
        controls = widgets.VBox([
            widgets.HBox([self.n_clusters]),
            widgets.HBox([self.play_button, self.slider, self.reset_button])
        ])
        display(widgets.VBox([self.output, self.info_output, controls]))
        self.update_plot()

# Create and run the interactive demo
InteractiveKMeans().run()

## I. K-means
K-means is an unsupervised machine learning algorithm used for clustering data into K groups. The goal is to find groups in the data, with the number of groups represented by the variable K.

## II. The K-means Algorithm

### A. Objective
The primary objective of K-means is to minimize the Within-Cluster Sum of Squares (WCSS), also known as inertia or Sum of Squared Errors (SSE).

### B. Steps of the Algorithm
1. **Initialization**: Randomly select K points as initial centroids.
2. **Assignment**: Assign each data point to the nearest centroid.
3. **Update**: Recalculate the centroids of the new clusters.
4. **Repeat**: Iterate steps 2 and 3 until convergence or a maximum number of iterations is reached.

### C. Pseudocode
```
function K-means(X, K):
    Initialize K centroids μ₁, μ₂, ..., μₖ randomly
    
    while not converged:
        // Assignment step
        for each x in X:
            Assign x to the cluster with the nearest centroid
        
        // Update step
        for i = 1 to K:
            μᵢ = mean of all points assigned to cluster i
    
    return centroids and cluster assignments
```

## III. Mathematical Foundations

### A. Dataset Representation
Let X = {x₁, x₂, ..., xₙ} be our dataset, where each xᵢ is a d-dimensional vector.

### B. Squared Euclidean Distance
The distance between two points x = (x₁, ..., xₘ) and y = (y₁, ..., yₘ) is calculated as:

$$d(x,y)^2 = \sum_{j=1}^m (x_j - y_j)^2 = \|x - y\|_2^2$$

This distance is used in the assignment step to determine the nearest centroid for each point.

### C. Sum of Squared Errors (SSE)
The objective function to be minimized is:

$$SSE = \sum_{i=1}^n \sum_{j=1}^k w^{(i,j)} \|\mathbf{X}^{(i)} - \boldsymbol{\mu}^{(j)}\|_2^2$$

Where:
- n is the number of data points
- k is the number of clusters
- w^(i,j) is a binary weight function (1 if point i is in cluster j, 0 otherwise)
- X^(i) is the i-th data point (vector)
- μ^(j) is the centroid of the j-th cluster (vector)

## IV. Algorithm Execution

### A. Initialization
Start with randomly chosen centroids μ₁, μ₂, ..., μₖ.

### B. Iteration
1. **Assignment Step**: 
   For each point xᵢ, calculate d(xᵢ, μⱼ)² for all j and assign xᵢ to the cluster with the smallest distance.

2. **Update Step**: 
   For each cluster j, recalculate μⱼ as the mean of all points assigned to it.

3. **SSE Calculation**: 
   Calculate the new SSE after the update.

4. **Convergence Check**: 
   If SSE hasn't significantly decreased or max iterations reached, stop. Otherwise, go back to step 1.

## V. Determining the Optimal K: The Elbow Method

### A. Purpose
The elbow method helps determine the optimal number of clusters (K) for K-means.

### B. Process
1. Run K-means for different values of K (e.g., K = 1 to 10).
2. For each K, calculate the SSE after convergence.
3. Plot K against SSE.
4. Look for the "elbow" point where the rate of decrease in SSE slows significantly.

### C. Interpretation
- The elbow point represents a good balance between the number of clusters and the SSE.
- It indicates the point of diminishing returns for adding more clusters.

### D. Limitations
- The elbow isn't always clear-cut.
- It may not always give the "correct" number of clusters for all applications.

## VI. Now w/ the elbow method

1. **Preparation**: 
   - Decide on a range of K values to try.
   - Prepare your dataset X.

2. **Elbow Method**:
   - For each K in your range:
     - Run K-means (steps II-IV)
     - Record the final SSE
   - Plot K vs SSE and identify the elbow point.

3. **Final Clustering**:
   - Choose K based on the elbow method and domain knowledge.
   - Run K-means with the chosen K to get your final clustering.

4. **Analysis**:
   - Examine the resulting clusters.
   - Interpret the centroids and cluster assignments in the context of your data.

## Our Implementation

In our interactive demo, we've implemented the K-means algorithm with the following key components:

1. **Synthetic Data Generation**: We create a 2D dataset with 5 distinct clusters to clearly demonstrate how K-means works.

2. **Initialization**: We randomly select K data points as initial centroids.

3. **Assignment**: In the `assign_labels` method, we calculate the Euclidean distance between each point and all centroids, assigning each point to the nearest centroid.

4. **Update**: The `update_centroids` method recalculates the centroid positions by taking the mean of all points in each cluster.

5. **Iteration**: The `step` method performs one iteration of assignment and update, allowing us to visualize the algorithm's progress.

6. **Visualization**: We plot the data points and centroids, updating in real-time to show how the clusters evolve.

7. **Statistics**: We calculate and display statistics for each cluster, including size and distances to the centroid, to provide insight into the clustering quality.


In [4]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure
import ipywidgets as widgets
from IPython.display import display
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class InteractiveKMeans:
    def __init__(self):
        self.fig = Figure(figsize=(10, 6))
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasAgg(self.fig)
        self.scatter = None
        self.centroid_scatter = None
        self.load_data()
        self.create_widgets()
        self.reset()

    def load_data(self):
        breast_cancer = load_breast_cancer()
        X = breast_cancer.data
        y = breast_cancer.target

        # Standardize the features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Apply PCA
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)

        self.data = X_pca
        self.true_labels = y
        self.explained_variance_ratio = pca.explained_variance_ratio_

    def create_widgets(self):
        self.play_button = widgets.Play(value=0, min=0, max=100, step=1, interval=1000, description="Play")
        self.slider = widgets.IntSlider(min=0, max=100, step=1, description='Iteration:')
        widgets.jslink((self.play_button, 'value'), (self.slider, 'value'))

        self.n_clusters = widgets.IntSlider(value=2, min=2, max=5, step=1, description='Clusters:')

        self.reset_button = widgets.Button(description="Reset")
        self.reset_button.on_click(self.reset)
        self.play_button.observe(self.step, 'value')
        
        self.output = widgets.Output()

    def reset(self, _=None):
        self.centroids = self.data[np.random.choice(len(self.data), self.n_clusters.value, replace=False)]
        self.labels = np.zeros(len(self.data), dtype=int)
        self.iteration = 0
        self.update_plot()
        self.play_button.value = 0

    def assign_labels(self):
        distances = np.sqrt(((self.data - self.centroids[:, np.newaxis])**2).sum(axis=2))
        self.labels = np.argmin(distances, axis=0)

    def update_centroids(self):
        for i in range(self.n_clusters.value):
            if np.sum(self.labels == i) > 0:
                self.centroids[i] = np.mean(self.data[self.labels == i], axis=0)

    def step(self, _):
        self.assign_labels()
        self.update_centroids()
        self.iteration += 1
        self.update_plot()

    def update_plot(self):
        self.ax.clear()
        self.scatter = self.ax.scatter(self.data[:, 0], self.data[:, 1], c=self.labels, cmap='viridis', alpha=0.7)
        self.centroid_scatter = self.ax.scatter(self.centroids[:, 0], self.centroids[:, 1], c='red', marker='x', s=200, linewidths=3)

        self.ax.set_xlabel(f'First Principal Component ({self.explained_variance_ratio[0]:.2%} variance)')
        self.ax.set_ylabel(f'Second Principal Component ({self.explained_variance_ratio[1]:.2%} variance)')
        self.ax.set_title(f'K-means Clustering on Breast Cancer Dataset (PCA) - Iteration {self.iteration}')

        self.canvas.draw()
        self.output.clear_output(wait=True)
        with self.output:
            display(self.fig)

    def run(self):
        controls = widgets.VBox([
            widgets.HBox([self.n_clusters]),
            widgets.HBox([self.play_button, self.slider, self.reset_button])
        ])
        display(widgets.VBox([self.output, controls]))
        self.update_plot()

# Create and run the interactive demo
InteractiveKMeans().run()

# K-means Clustering on Breast Cancer Data: Beyond the Algorithm

## Introduction to the Dataset

The breast cancer dataset is a classic example used in machine learning. It contains features computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. These features describe characteristics of the cell nuclei present in the image.

## Applying K-means to Medical Data

When we apply K-means clustering to this dataset, we're essentially trying to group patients based on similarities in their cell nuclei characteristics. However, this application raises several important considerations:

1. **Dimensionality Reduction**: Our code uses PCA to reduce the dataset's dimensionality. This is crucial because:
   - It allows us to visualize the data in 2D.
   - It can help identify the most important features.
   - It may improve clustering performance by reducing noise.

2. **Standardization**: We standardize the features before applying PCA and K-means. This is important because:
   - Features are measured on different scales (e.g., area, texture, perimeter).
   - K-means is sensitive to the scale of the features.

3. **Choosing K**: In our interactive demo, we allow K to vary from 2 to 5. In reality, choosing the right number of clusters is a complex decision that often requires domain expertise.

## Challenges in Real-World Clustering

Applying K-means to real-world medical data like this presents several challenges:

1. **Interpretability**: While we might find clusters, interpreting what they mean medically is crucial. A cluster doesn't inherently mean "benign" or "malignant".

2. **Overlapping Characteristics**: In real medical data, the line between different conditions is often blurry. Clusters may not be as well-defined as we hope.

3. **Hidden Variables**: There might be important factors not captured in our dataset that influence the true grouping of patients.

4. **Non-Spherical Clusters**: K-means assumes clusters are spherical and equally sized, which may not reflect the true structure of the data.

5. **Outliers and Noise**: Medical data often contains outliers or measurement errors which can significantly impact K-means results.

## The Crucial Role of Domain Expertise
In the realm of medical data analysis, the input of healthcare professionals is indispensable. Their expertise plays a pivotal role in various aspects of the clustering process. From the outset, domain experts can guide the feature selection process, helping to identify which attributes are most likely to have clinical relevance. Once clusters are discovered, medical professionals provide the necessary context to interpret these groupings from a clinical perspective. Their insight is equally crucial in the validation phase, where they can assess whether the discovered patterns align with medical knowledge and make sense from a practical standpoint. Moreover, healthcare experts are instrumental in navigating the ethical landscape, offering guidance on the implications of using clustering results in patient care decisions.

## Beyond K-means: A Data Scientist's Perspective
As data scientists working in the healthcare sector, we must approach clustering with a nuanced understanding of its role and limitations. It's essential to view clustering results as exploratory findings that generate hypotheses for further investigation, rather than definitive categorizations. To enhance the robustness of our analysis, K-means should be considered as one tool in a broader analytical toolkit, potentially combined with other techniques such as hierarchical clustering or density-based methods. While visualizations like 2D PCA plots are valuable for demonstration purposes, we must exercise caution to avoid oversimplifying complex medical data. Finally, when presenting our findings to medical professionals, it's our responsibility to clearly articulate the uncertainties, limitations, and underlying assumptions of our analysis, ensuring that our results are interpreted within the appropriate context.